In [6]:
import os
import requests
from typing import Type
from pydantic import BaseModel, Field
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper

llm = ChatOpenAI(
    temperature=0.1,
    model_name="gpt-3.5-turbo"
)
alpha_vantage_api_key = os.environ.get("ALPHA_VANTAGE_API_KEY")

In [11]:
class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="The query you will search for.Example query: Stock Market Symbol for Apple Company"
    )


class StockMarketSymbolSearchTool(BaseTool):
    name = "StockMarketSymbolSearchTool"
    description = """
    Use this tool to find the stock market symbol for a company.
    It takes a query as an argument.
    """
    args_schema: Type[StockMarketSymbolSearchToolArgsSchema] = StockMarketSymbolSearchToolArgsSchema

    def _run(self, query: str):
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)


class CompanyOverviewArgsSchema(BaseModel):
    symbol: str = Field(
        description="stock symbol of the company.Example: AAPL, TSLA"
    )


class CompanyOverviewTool(BaseTool):
    name = "CompanyOverview"
    description = """
    Use this to get an overview of the financials of the company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        return r.json()


class CompanyIncomeStatementTool(BaseTool):
    name = "CompanyIncomeStatement"
    description = """
    Use this to get the income statement of a company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        return r.json()["annualReports"]


class CompanyStockPerformanceTool(BaseTool):
    name = "CompanyStockPerformance"
    description = """
    Use this to get the weekly performance of a company stock.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        response = r.json()
        return list(response["Weekly Time Series"].items())[:200]


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        StockMarketSymbolSearchTool(),
        CompanyOverviewTool(),
        CompanyIncomeStatementTool(),
        CompanyStockPerformanceTool(),
    ],
)

prompt = ("Give me financial information on tesla's stock, considering its financials, income statements and stock "
          "performance help me analyze if it's a potential good investment.")

agent.invoke(prompt)





> Entering new AgentExecutor chain...

Invoking: `StockMarketSymbolSearchTool` with `{'query': 'Stock Market Symbol for Tesla'}`


Discover real-time Tesla, Inc. Common Stock (TSLA) stock prices, quotes, historical data, news, and Insights for informed trading and investment decisions. Stay ahead with Nasdaq. The latest Tesla stock prices, stock quotes, news, and TSLA history to help you invest and trade smarter. ... Tesla's market capitalization is $544.61 B by 3.18 B shares outstanding. TSLA Competitors. $ Market cap P/E ratio $ Price 1d change 52-week range. View Tesla, Inc. TSLA stock quote prices, financial information, real-time forecasts, and company news from CNN. Stock analysis for Tesla Inc (TSLA:NASDAQ GS) including stock price, stock chart, company news, key statistics, fundamentals and company profile. Tesla TSLA quarterly deliveries declined by 8.5% from a year ago to 386,810 vehicles for Q1 2024 and fell well below Wall Street estimates. The number was also down 20.2% 

{'input': "Give me financial information on tesla's stock, considering its financials, income statements and stock performance help me analyze if it's a potential good investment.",
 'output': "### Tesla Inc. Financial Information:\n\n#### Company Overview:\n- **Symbol:** TSLA\n- **Market Capitalization:** $547.02 billion\n- **PE Ratio:** 39.94\n- **EPS:** 4.3\n- **Revenue (TTM):** $96.77 billion\n- **Profit Margin:** 15.5%\n- **Operating Margin (TTM):** 8.2%\n- **Return on Assets (TTM):** 5.88%\n- **Return on Equity (TTM):** 27.4%\n- **Sector:** Manufacturing\n- **Industry:** Motor Vehicles & Passenger Car Bodies\n- **Analyst Target Price:** $188.42\n- **52-Week High:** $299.29\n- **52-Week Low:** $152.37\n\n#### Income Statement (2023):\n- **Total Revenue:** $96.77 billion\n- **Gross Profit:** $17.66 billion\n- **Operating Income:** $8.89 billion\n- **Net Income:** $14.97 billion\n\n### Analysis:\n- Tesla has shown consistent revenue growth over the years, with a strong market capita